In [8]:
from pyfuseki import FusekiQuery, FusekiUpdate
from pysolr import Solr
import httpx
from rdflib import Graph
from api.src.routes.translate.makeTranslate import MakeTranslate
from api.src.function.loc.graphExist import GraphExist
from pydantic import BaseModel
from typing import Optional
from api.src.schemas.authorities.authority import Uri, Variant, AdminMetadata, Element

In [28]:
au_update = FusekiUpdate('http://localhost:3030', 'authorities')

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = au_update.run_sparql(d)
response.convert()

solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)
solr.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">134</int>\n</lst>\n</response>\n'

Information science

In [4]:
url = 'http://localhost:8000/authorities/loc/subject?uri=http://id.loc.gov/authorities/subjects/sh85066150'
r = httpx.get(url)
r.json()

{'detail': 'Esse registro já existe'}

In [30]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/authorities/mads/subject/', headers=headers, json=r.json())
response.status_code

201

In [29]:
graph = Graph()
uri = 'http://id.loc.gov/authorities/subjects/sh85066150'
graph.parse(f'{uri}.rdf')
graph.serialize('information.ttl')

<Graph identifier=Nf18366b49afd41bd82bac6c384eacfb0 (<class 'rdflib.graph.Graph'>)>

In [18]:
fuseki_query = FusekiQuery('http://localhost:3030', 'authorities')


class Subject(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    note: Optional[str]
    hasBroaderAuthority: Optional[list[Uri]]
    hasBroaderExternalAuthority: Optional[list[Uri]]
    hasNarrowerAuthority: Optional[list[Uri]]
    hasNarrowerExternalAuthority: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str

def GetUriBK(url):
    token = url.split("/")[-1]
    queryBK = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
      PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
      SELECT ?uri ?label WHERE {{ graph ?g {{
          ?uri madsrdf:authoritativeLabel ?label .
        ?sub bf:identifiedBy ?identifiedBy .
          ?identifiedBy rdf:value "{token}"
          FILTER EXISTS {{ ?identifiedBy bf:assigner <http://id.loc.gov/vocabulary/organizations/dlc> }}
      }} }} """
    response = fuseki_query.run_sparql(queryBK)
    [binding] = response.convert()['results']['bindings']
    uri = {"value": binding['uri']['value'],
              "label": {
                "value": binding['label']['value'],
                "lang": "pt"
              } }
    return uri

def GetUri(obj, metadado, authority, graph):
    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:{metadado} ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        uris = list()
        for i in response.bindings:
          url = i.get('value')
          if metadado in ['hasBroaderAuthority', 'hasNarrowerAuthority']:
            exist = GraphExist(url)
            if exist:
              uri = GetUriBK(url)
            else:
              uri = {
                    "value": url,
                    "label": {
                  "value": i.get('label').value,
                  "lang": i.get('label').language
                } }
          else:
            uri = {
                  "value": url,
                   "label": {
                "value": i.get('label').value,
                "lang": i.get('label').language
              } }
          uris.append(uri)
        obj[metadado] = uris
    return obj



In [14]:
def GetHasBroader(authority, graph, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:hasBroaderAuthority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
           url = i.get('value')
           exist = GraphExist(url)
           if exist:
               print(exist)
           else:
               uri = {
                   "value": url,
                   "base": "loc",
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
               external.append(uri)
        if len(external) > 0:
            obj['hasBroaderExternalAuthority'] = external
    return obj


    
    

In [26]:
def GetHasNarrower(authority, graph, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:hasNarrowerAuthority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
           url = i.get('value')
           exist = GraphExist(url)
           if exist:
               print(exist)
           else:
               uri = {
                   "value": url,
                   "base": "loc",
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
               external.append(uri)
        if len(external) > 0:
            obj['hasNarrowerExternalAuthority'] = external
    return obj
obj = {}
obj = GetHasNarrower(uri, graph, obj)
obj

{'hasNarrowerExternalAuthority': [{'value': rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh85029072'),
   'base': 'loc',
   'label': {'value': 'Communication in information science', 'lang': 'en'}},
  {'value': rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh99004272'),
   'base': 'loc',
   'label': {'value': 'Geomatics', 'lang': 'en'}},
  {'value': rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh97004947'),
   'base': 'loc',
   'label': {'value': 'Agricultural informatics', 'lang': 'en'}},
  {'value': rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh85038731'),
   'base': 'loc',
   'label': {'value': 'Documentation', 'lang': 'en'}},
  {'value': rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh85066148'),
   'base': 'loc',
   'label': {'value': 'Information retrieval', 'lang': 'en'}},
  {'value': rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh85066157'),
   'base': 'loc',
   'label': {'value': 'Information se

In [27]:
def ParserSubject(graph, authority):
      
  prefix = """PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>"""

  # Type
  qtype = f"""{prefix}
  SELECT ?type WHERE {{ 
      <{authority}> rdf:type ?type .
      FILTER ( ?type != madsrdf:Authority ) 
      FILTER ( ?type != skos:Concept )
       }}"""
  r = graph.query(qtype)
  tipo = r.bindings[0].get('type').split("#")[1]
  

  # adminMetadata
  adminMetadata = {
      "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
      "identifiedBy": [ {
         "type": "Lccn",
          "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
          "value": authority.split('/')[-1]        
      }]      
  }
  # ElementList
  qElementList = f"""{prefix}
  SELECT ?elementValue ?type WHERE  {{
    <{authority}> madsrdf:elementList ?o .
    ?o rdf:rest* ?node .
    ?node rdf:first ?e .
    ?e madsrdf:elementValue ?elementValue .
    ?e rdf:type ?type
    }}"""
  r = graph.query(qElementList)
  elementList = list()
  for i in r.bindings:
      element = {
            "type": i.get('type').split("#")[1],
          "elementValue": {
            "value":  i.get('elementValue').value,
            "lang": i.get('elementValue').language
          }
        }
      elementList.append(element)

  obj = {
     "type": tipo,
      "adminMetadata": adminMetadata,
      "elementList": elementList,
      "isMemberOfMADSCollection": f'http://bibliokeia.com/authorities/{tipo}/'
  }

  # Note 
  qNote = f"""{prefix}
  SELECT ?note WHERE {{ 
      <{authority}> madsrdf:note ?note .
       }}"""
  r = graph.query(qNote)
  if len(r.bindings) > 0:
     obj['note'] = r.bindings[0].get('note').value

# hasVariant
  qVariant = f"""{prefix}
  SELECT ?typeVariant ?typeElement ?elementValue WHERE  {{
	<{authority}> madsrdf:hasVariant ?variant .
  ?variant rdf:type ?typeVariant .
  ?variant madsrdf:elementList ?elementList .
  ?elementList rdf:rest* ?node .
    ?node rdf:first ?e .
    ?e madsrdf:elementValue ?elementValue .
	?e rdf:type ?typeElement .
  FILTER ( ?typeVariant != madsrdf:Variant )
  }}"""
  r = graph.query(qVariant)
  if len(r.bindings) > 0:
    variants = list()
    for i in r.bindings:
      variant = {
          'type': i.get('typeVariant').split("#")[1],
          'elementList': [{
              'type': i.get('typeElement').split("#")[1],
              'elementValue': {
                  'value': i.get('elementValue').value,
                  'lang': i.get('elementValue').language
              }
          }]
      }
      variants.append(variant)
    obj['hasVariant'] = variants

  # URIS
  # metadados = ['hasBroaderAuthority', 
  # 'hasNarrowerAuthority',
  # 'hasCloseExternalAuthority',
  #   'hasExactExternalAuthority',
  #   'hasNarrowerExternalAuthority']
  # for metadado in metadados:
  #    obj = GetUri(obj, metadado, authority, graph)
  obj = GetHasBroader(authority, graph, obj)
  # Narrower Terms
  obj = GetHasNarrower(authority, graph, obj)
  # hasReciprocalAuthority
  
  response = Subject(**obj)
  return response

In [28]:
request = ParserSubject(graph, uri)
request.dict()

{'type': 'Topic',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': datetime.date(2023, 5, 11),
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-05-11T16:13:16',
  'identifiedBy': [{'type': 'Lccn',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': 'sh85066150'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'elementList': [{'type': 'TopicElement',
   'elementValue': {'value': 'Information science', 'lang': 'en'}}],
 'note': None,
 'hasBroaderAuthority': None,
 'hasBroaderExternalAuthority': [{'value': rdflib.term.URIRef('http://id.loc.gov/authorities/subjects/sh85029027'),
   'label': {'value': 'Communication', 'lang': 'en'},
   'base': 'loc'}],
 'hasNarrowerAuthority': None,
 'hasCloseExternalAuthority': None,
 'hasExactExternalAu

In [27]:

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}
response = httpx.post('http://localhost:8000/authorities/mads/subject/', headers=headers, json=r.json())
response.status_code

201